# NEMA Photoelectric

- Photoelectric interaction in PETALO

In [1]:
import Pkg

In [2]:
Pkg.add.(["HTTP", "Plots", "DataFrames", "HDF5","DrWatson", "StatsBase", "PhysicalConstants"]);

In [3]:
Pkg.add.(["Test"]);

In [4]:
Pkg.add.(["VegaLite","VegaDatasets"]);

In [5]:
Pkg.add.(["LsqFit","GLM"])

2-element Vector{Nothing}:
 nothing
 nothing

In [6]:
Pkg.add("Distributions") 

In [7]:
using Distributions

In [8]:
using LsqFit
using GLM

In [9]:
using HTTP
using Plots
using VegaLite
using VegaDatasets
using DataFrames
using HDF5
using DrWatson
using Test
using PhysicalConstants
using PhysicalConstants.CODATA2018
using Unitful

In [10]:
using Glob
using CSV

In [11]:
using Statistics
using StatsBase
using Printf

In [12]:
using Logging

In [13]:
@quickactivate "JPetalo"

In [14]:
pwd()

"/Users/jj/JuliaProjects/JPetalo/notebooks"

In [15]:
datadir()

"/Users/jj/JuliaProjects/JPetalo/data"

In [16]:
datadir("nema3f")

"/Users/jj/JuliaProjects/JPetalo/data/nema3f"

In [17]:
srcdir()

"/Users/jj/JuliaProjects/JPetalo/src"

In [18]:
include(srcdir("jpetalo.jl"))

Main.JPetalo

In [19]:
import Unitful:
    nm, μm, mm, cm, m, km, inch, ft, mi,
    ac,
    mg, g, kg,
    Ra, °F, °C, K,
    rad, °,
    ns, μs, ms, ps, s, minute, hr, d, yr, Hz,
    eV,
    μJ, mJ, J,
	mW, μW, W,
    A, N, mol, mmol, V, L, mL, μL, M

In [20]:
function loglevel(log)
    if log == "Debug"
        logger = SimpleLogger(stdout, Logging.Debug)
    elseif log =="Info"
        logger = SimpleLogger(stdout, Logging.Info)
    else
        logger = SimpleLogger(stdout, Logging.Warn)
    end
    old_logger = global_logger(logger)
end

loglevel (generic function with 1 method)

# Characterization of PETALO

In [21]:
loglevel("Info")

Base.CoreLogging.SimpleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(44) open, 0 bytes waiting))), Info, Dict{Any, Int64}())

### Read a summary data frame with photoelectric interactions

In [34]:
drx = datadir("nema3df")
input = string(drx,"/nemadf_f600_pde_03_sigmatof_85ps_kmeans_phot.csv")
evtdf = DataFrame(CSV.File(input));

In [35]:
@info names(evtdf)

┌ Info: ["nsipm1", "nsipm2", "q1", "q2", "r1", "r1q", "r2", "r2q", "t1", "t2", "ta1", "ta2", "tr1", "tr2", "ux", "uy", "uz", "x1", "x2", "xb1", "xb2", "xr1", "xr2", "xs", "xt1", "xt2", "y1", "y2", "yb1", "yb2", "yr1", "yr2", "ys", "yt1", "yt2", "z1", "z2", "zb1", "zb2", "zr1", "zr2", "zs", "zt1", "zt2"]
└ @ Main In[35]:1


In [24]:
function setunits(df::DataFrame)
    DataFrame(
        nsipm1 = df.nsipm1,     # number of sipms in cluster
        nsipm2 = df.nsipm2,
        
        q1 = df.q1,             # q in pes
        q2 = df.q2,
        
        r1  = df.r1  * mm,      # best radius
        r1q = df.r1q * mm,      # raw radius from q 
        r2  = df.r2  * mm,
        r2q = df.r2q * mm,
        
        t1  = df.t1  * ns,      # time true
        t2  = df.t2  * ns,
        ta1 = df.ta1 * ns,      # average (over 5 mimimum in t)
        ta2 = df.ta2 * ns,
        tr1 = df.tr1 * ns,      # reco (smeared take minimum)
        tr2 = df.tr2 * ns,
        
        ux = df.ux,             # unit direction vector of gammas 
        uy = df.uy,
        uz = df.uz,
        
        x1  = df.x1  * mm,      # best reco position
        x2  = df.x2  * mm,
        xb1 = df.xb1 * mm,      # position of sipm that gives time stamp
        xb2 = df.xb2 * mm,
        xr1 = df.xr1 * mm,      # reco position
        xr2 = df.xr2 * mm,
        xs  = df.xs  * mm,      # position of source
        xt1 = df.xt1 * mm,      # true position
        xt2 = df.xt2 * mm,
        
        y1  = df.y1  * mm,      
        y2  = df.y2  * mm,
        yb1 = df.yb1 * mm,      
        yb2 = df.yb2 * mm,
        yr1 = df.yr1 * mm,      
        yr2 = df.yr2 * mm,
        ys  = df.ys  * mm,
        yt1 = df.yt1 * mm,      
        yt2 = df.yt2 * mm,
        
        z1  = df.z1  * mm,      
        z2  = df.z2  * mm,
        zb1 = df.zb1 * mm,      
        zb2 = df.zb2 * mm,
        zr1 = df.zr1 * mm,      
        zr2 = df.zr2 * mm,
        zs  = df.zs  * mm,
        zt1 = df.zt1 * mm,      
        zt2 = df.zt2 * mm,
        
    )
end

setunits (generic function with 1 method)

In [36]:
evtdfu = setunits(evtdf);

In [37]:
first(evtdfu, 5)

,nsipm1,nsipm2,q1,q2,r1,r1q,r2,r2q
,Int64,Int64,Float64,Float64,Quantit…,Quantit…,Quantit…,Quantit…
1,241,160,2095.0,2459.8,368.295 mm,368.565 mm,381.841 mm,378.415 mm
2,260,260,1876.4,1952.5,359.837 mm,362.663 mm,363.159 mm,364.717 mm
3,254,247,1807.1,1819.5,359.066 mm,360.792 mm,360.022 mm,361.127 mm
4,211,255,2163.7,2044.6,372.581 mm,370.42 mm,365.835 mm,367.204 mm
5,244,225,1934.4,2057.2,364.667 mm,364.229 mm,369.897 mm,367.544 mm


In [38]:
@enum Dtsel dtfirst dtminimum dtaverage

In [39]:
@enum Possel postrue posreco

In [40]:
function deltatime(df::DataFrame, t::Dtsel=dtfirst)
    if t     == dtminimum
        return  df.tr1 - df.tr2
    elseif t == dtaverage
        return df.ta1 - df.ta2
    else
        return df.t1 - df.t2

    end
end

deltatime (generic function with 2 methods)

In [47]:
function cdoi(df::DataFrame, position::Possel=postrue, nlxe::Number=1.6)
    clxe = SpeedOfLightInVacuum/nlxe

    if position == posreco
        dxrb1 = [JPetalo.dxyz([df.x1[i], df.y1[i], df.z1[i]],
                              [df.xb1[i], df.yb1[i], df.zb1[i]]) for i in 1:nrow(df)]

        dxrb2 = [JPetalo.dxyz([df.x2[i], df.y2[i], df.z2[i]],
                              [df.xb2[i], df.yb2[i], df.zb2[i]]) for i in 1:nrow(df)]
    else
        dxrb1 = [JPetalo.dxyz([df.xt1[i], df.yt1[i], df.zt1[i]],
                              [df.xb1[i], df.yb1[i], df.zb1[i]]) for i in 1:nrow(df)]

        dxrb2 = [JPetalo.dxyz([df.xt2[i], df.yt2[i], df.zt2[i]],
                              [df.xb2[i], df.yb2[i], df.zb2[i]]) for i in 1:nrow(df)]

    end
    return (dxrb1 - dxrb2)/clxe
end

cdoi (generic function with 3 methods)

In [42]:
struct MlemLor
    dx::Float32
    x1::Float32
    y1::Float32
    z1::Float32
    x2::Float32
    y2::Float32
    z2::Float32
end

In [43]:
function dftolor(df::DataFrame, t::Dtsel=dtfirst, position::Possel=postrue, nlxe::Number=1.6)

    function tof32(l)
        Float32.(l/mm)
    end

    dt12  = deltatime(df,t)
    dtdoi = cdoi(df,position)
    # compute dx from time and speed of light, ensure that the result is in mm
    dx    = uconvert.(mm, (dt12 - dtdoi) * SpeedOfLightInVacuum)

    if position == postrue
        x1, x2, y1, y2, z1, z2 = df.xt1, df.xt2, df.yt1, df.yt2, df.zt1, df.zt2
    else
        x1, x2, y1, y2, z1, z2 = df.x1,  df.x2,  df.y1,  df.y2,  df.z1,  df.z2
    end

    MlemLor.(tof32(dx),tof32(x1),tof32(y1),tof32(z1), tof32(x2), tof32(y2), tof32(z2))
end


dftolor (generic function with 4 methods)

In [44]:
dt12 = deltatime(evtdfu,dtfirst);

In [45]:
length(dt12)

32543

In [48]:
dtdoi = cdoi(evtdfu,postrue);

In [49]:
length(dtdoi)

32543

In [50]:
function write_lors_hdf5(filename, mlor)

    function set_datatype(::Type{MlemLor})
        dtype = HDF5.h5t_create(HDF5.H5T_COMPOUND, sizeof(MlemLor))
        HDF5.h5t_insert(dtype, "dx", fieldoffset(MlemLor, 1),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "x1", fieldoffset(MlemLor, 2),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "y1", fieldoffset(MlemLor, 3),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "z1", fieldoffset(MlemLor, 4),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "x2", fieldoffset(MlemLor, 5),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "y2", fieldoffset(MlemLor, 6),
                        datatype(Float32))
        HDF5.h5t_insert(dtype, "z2", fieldoffset(MlemLor, 7),
                        datatype(Float32))

        HDF5.Datatype(dtype)
    end

    h5f = JPetalo.h5open(filename, "w")

    dtype = set_datatype(MlemLor)
    dspace = dataspace(mlor)
    grp = create_group(h5f, "true_info")
    dset = create_dataset(grp, "lors", dtype, dspace)
    write_dataset(dset, dtype, mlor)


    close(h5f)
end


write_lors_hdf5 (generic function with 1 method)

In [51]:
lortrue = dftolor(evtdfu, dtfirst, postrue);

In [52]:
lortrue[1]

MlemLor(3.4322562f0, 38.4391f0, -366.28372f0, -16.189302f0, -37.79373f0, 379.96616f0, 214.23856f0)

In [53]:
write_lors_hdf5(datadir("lors/lortrue.h5"), lortrue)

In [91]:
function df_to_mlemlor(ldf::DataFrame)
    [JPetalo.MlemLor(ldf.t1[i], ldf.t2[i],
                  ldf.x1[i],ldf.y1[i],ldf.z1[i],
                  ldf.x2[i],ldf.y2[i],ldf.z2[i]) for i in 1:nrow(ldf)]
end

xxx (generic function with 1 method)